#### Produce constructor standings

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

In [0]:
dbutils.widgets.text("p_file_date", "2021-03-21")
v_file_date = dbutils.widgets.get("p_file_date")
v_file_date

In [0]:
race_year_list = []
for race_year in race_results_list:
    race_year_list.append(race_year.race_year)

In [0]:
path = f"{presentation_folder_path}/race_results"
race_results_df = column_to_list(path, 'race_year',v_file_date)
display(race_results_df)

In [0]:
from pyspark.sql.functions import sum, count, when, col

In [0]:
constructor_standings_df = race_results_df \
.groupBy("race_year", "team") \
.agg(sum("points").alias("total_points"), count(when(col("position") == 1, True)).alias("wins"))
display(constructor_standings_df.filter("race_year = 2020"))

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import desc, rank

In [0]:
constructor_rank_spec = Window.partitionBy("race_year").orderBy(desc("total_points"), desc("wins"))
final_df = constructor_standings_df.withColumn("rank", rank().over(constructor_rank_spec))

In [0]:
# display(final_df.filter("race_year = 2020"))

In [0]:
# final_df.write.mode("overwrite").parquet(f"{presentation_folder_path}/constructor_standings")
# final_df.write.mode("overwrite").format("parquet").saveAsTable("f1_presentation.constructor_standings")
overwrite_partition(final_df, 'f1_presentation', 'constructor_standings', 'race_year')